In [ ]:
// Configure the connection to the data lake
var adlsPath = "abfss://wwi@<primary_storage>.dfs.core.windows.net";

In [ ]:
// Import the sales data from the data lake
var sales = spark.Read().Option("header", "true").Option("inferSchema", "true").Option("sep", "|").Csv($"{adlsPath}/factsale-csv/2012/Q4");
sales.Show(5);
sales.PrintSchema();

In [ ]:
using System;
using System.Collections.Generic;

// Set up a JDBC connection to Azure Synapse Analytics
var jdbcUsername = "asa.sql.staging@<workspace>";
var jdbcPassword = "<sql_staging_password>";
var jdbcConnectionString = $"jdbc:sqlserver://<workspace>.sql.azuresynapse.net:1433;database=<sql_pool>;";

// Set target table name
var tableName = $"wwi_staging.Sale_CSharp";

// Write the retrieved sales data into a staging table in Azure Synapse Analytics.
sales
    .Limit(1500)
    .Write()
    .Format("jdbc")
    .Options(
        new Dictionary<string, string>
        {
            {"url", jdbcConnectionString},
            {"dbtable", tableName},
            {"user", jdbcUsername},
            {"password", jdbcPassword},
            {"encrypt", "false"},
            {"trustServerCertificate", "true"},
            {"createTableColumnTypes", "Description VARCHAR(100), Package VARCHAR(50)"} // Override the column types for string values. They are assigned nvarchar(max) by default, and that is not compatible with columnstore index.
        }
    )
    .Mode("append")
    .Save();